In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
from scipy.stats import norm, skew # 수치해석 기능(확률분포) 제공
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,f1_score,roc_curve,roc_auc_score,log_loss

train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/천체 유형 분류/train.csv')
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/천체 유형 분류/test.csv')
sample_submission = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/천체 유형 분류/sample_submission.csv')

train_id = train['id']
test_id = test['id']

# 필요없는 Id 컬럼, Drop하자.
#train.drop('id',axis=1,inplace=True)
#test.drop('id',axis=1,inplace=True)

# 'Id'를 제거한 DataFrame의 Shape 확인
print("The train data size after dropping ID feature is : {} ".format(train.shape))
print("The test data size after dropping Id feature is : {} ".format(test.shape))

unique_labels = train['type'].unique()
label_dict = {val: i for i, val in enumerate(unique_labels)}
i2lb = {v:k for k, v in label_dict.items()}

The train data size after dropping ID feature is : (199991, 23) 
The test data size after dropping Id feature is : (10009, 22) 


In [0]:
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x: to_number(x, column_number))

In [0]:
test_id=test['id']
test = test.drop(columns=['fiberID', 'id'])

train_x = train.drop(columns=['id','type_num', 'fiberID','type'], axis=1)
train_y = train['type_num']
test_x = test

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train_x, train_y, \
                                                  test_size=0.2, random_state=42, stratify = train_y)

# LGBM
- test logloss: 0.39

In [0]:
# #LGBM 1

# from lightgbm import LGBMClassifier

# lgbm_wrapper = LGBMClassifier(boosting_type='gbdt', num_leaves=250, max_depth=-1, learning_rate=0.005, min_data_in_leaf = 5,
#                              n_estimators=10000, subsample_for_bin=250000, objective='multiclass', min_split_gain=0.34, reg_alpha = 0.01, reg_lambda = 0.01,
#                              min_child_weight=0.005, min_child_samples=17,n_jobs=-1, random_state=123)

# evals =[(X_train,y_train),(X_valid,y_valid)]
# lgbm_wrapper.fit(X_train,y_train,early_stopping_rounds=20,
#                  eval_metric='logloss',eval_set=evals,verbose=True)

# #- test logloss: 0.39

In [25]:
#LGBM 1

from lightgbm import LGBMClassifier

lgbm_wrapper = LGBMClassifier(boosting_type='gbdt', num_leaves=500, max_depth=-1, learning_rate=0.05, min_data_in_leaf = 5,
                             n_estimators=10000, subsample_for_bin=250000, objective='multiclass', min_split_gain=0.34, reg_alpha = 0.01, reg_lambda = 0.01,
                             min_child_weight=0.005, min_child_samples=17,n_jobs=-1, random_state=123)

evals =[(X_train,y_train),(X_valid,y_valid)]
lgbm_wrapper.fit(X_train,y_train,early_stopping_rounds=20,
                 eval_metric='logloss',eval_set=evals,verbose=True)

#- test logloss: 0.39

[1]	training's multi_logloss: 2.00587	training's multi_logloss: 2.00587	valid_1's multi_logloss: 2.04668	valid_1's multi_logloss: 2.04668
Training until validation scores don't improve for 20 rounds.
[2]	training's multi_logloss: 1.83871	training's multi_logloss: 1.83871	valid_1's multi_logloss: 1.89375	valid_1's multi_logloss: 1.89375
[3]	training's multi_logloss: 1.7063	training's multi_logloss: 1.7063	valid_1's multi_logloss: 1.77366	valid_1's multi_logloss: 1.77366
[4]	training's multi_logloss: 1.59472	training's multi_logloss: 1.59472	valid_1's multi_logloss: 1.67248	valid_1's multi_logloss: 1.67248
[5]	training's multi_logloss: 1.49784	training's multi_logloss: 1.49784	valid_1's multi_logloss: 1.58428	valid_1's multi_logloss: 1.58428
[6]	training's multi_logloss: 1.41216	training's multi_logloss: 1.41216	valid_1's multi_logloss: 1.50631	valid_1's multi_logloss: 1.50631
[7]	training's multi_logloss: 1.33544	training's multi_logloss: 1.33544	valid_1's multi_logloss: 1.43661	valid_1

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.05, max_depth=-1,
               min_child_samples=17, min_child_weight=0.005, min_data_in_leaf=5,
               min_split_gain=0.34, n_estimators=10000, n_jobs=-1,
               num_leaves=500, objective='multiclass', random_state=123,
               reg_alpha=0.01, reg_lambda=0.01, silent=True, subsample=1.0,
               subsample_for_bin=250000, subsample_freq=0)

## LGBM voting

In [0]:
from lightgbm import LGBMClassifier

#############

lgbm_wrapper1 = LGBMClassifier(boosting_type='gbdt', num_leaves=250, max_depth=-1, learning_rate=0.005, min_data_in_leaf = 5,
                             n_estimators=10000, subsample_for_bin=250000, objective='multiclass', min_split_gain=0.34, reg_alpha = 0.01, reg_lambda = 0.01,
                             min_child_weight=0.005, min_child_samples=17,n_jobs=-1, random_state=123)
###############

lgbm_wrapper2 = LGBMClassifier(boosting_type='gbdt', num_leaves=300, max_depth=-1, learning_rate=0.005, min_data_in_leaf = 5,
                             n_estimators=10000, subsample_for_bin=250000, objective='multiclass', min_split_gain=0.34, reg_alpha = 0.01, reg_lambda = 0.01,
                             min_child_weight=0.005, min_child_samples=17,n_jobs=-1, random_state=123)
#################

lgbm_wrapper3 = LGBMClassifier(boosting_type='gbdt', num_leaves=350, max_depth=-1, learning_rate=0.005, min_data_in_leaf = 5,
                             n_estimators=10000, subsample_for_bin=250000, objective='multiclass', min_split_gain=0.34, reg_alpha = 0.01, reg_lambda = 0.01,
                             min_child_weight=0.005, min_child_samples=17,n_jobs=-1, random_state=123)

In [29]:
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(
    estimators=[('lgb1', lgbm_wrapper1), ('lgb2', lgbm_wrapper2), ('lgb3', lgbm_wrapper3)],
    voting='soft')

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lgb1',
                              LGBMClassifier(boosting_type='gbdt',
                                             class_weight=None,
                                             colsample_bytree=1.0,
                                             importance_type='split',
                                             learning_rate=0.005, max_depth=-1,
                                             min_child_samples=17,
                                             min_child_weight=0.005,
                                             min_data_in_leaf=5,
                                             min_split_gain=0.34,
                                             n_estimators=10000, n_jobs=-1,
                                             num_leaves=250,
                                             objective='multiclass',
                                             random_state=123, reg_alpha=0.01...
                                             learning_rate=0

In [31]:
yk_grd_probs = voting_clf.predict_proba(X_valid)
print(log_loss(y_valid, yk_grd_probs))

0.39204467979189705


# Submission

In [0]:
preds = voting_clf.predict_proba(test)

In [0]:
submission = pd.DataFrame(preds)
submission = pd.concat([test_id, submission], axis=1)
submission.columns=['id', 'STAR_WHITE_DWARF', 'STAR_CATY_VAR', 'STAR_BROWN_DWARF',
       'SERENDIPITY_RED', 'REDDEN_STD', 'STAR_BHB', 'GALAXY',
       'SERENDIPITY_DISTANT', 'QSO', 'SKY', 'STAR_RED_DWARF', 'ROSAT_D',
       'STAR_PN', 'SERENDIPITY_FIRST', 'STAR_CARBON', 'SPECTROPHOTO_STD',
       'STAR_SUB_DWARF', 'SERENDIPITY_MANUAL', 'SERENDIPITY_BLUE']

submission.to_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/천체 유형 분류/submission_lgbm_Voting.csv')